In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [2]:
import yaml
from easydict import EasyDict as edict

In [3]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [4]:
def plot_lambda_raster(im_o, im_t):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(90, 80))
    time = im_o.shape[0] / 10000
    extent = [0, time, 0, 100]
    im_o = axes[0].imshow(np.transpose(im_o), cmap='binary', extent=extent)
    im_t = axes[1].imshow(np.transpose(im_t), cmap='binary', extent=extent)
    ylabel = 'Lambda'
    axes[0].set_title('Prediction\n', fontsize = 30)
    axes[0].set_xlabel('Time(s)', fontsize = 30)
    axes[0].set_ylabel('Node index', fontsize = 30)
    axes[0].set_aspect(0.1)
    axes[0].tick_params(axis='x', labelsize=20)
    axes[0].tick_params(axis='y', labelsize=20)
    axes[1].set_title('Target\n', fontsize = 30)
    axes[1].set_xlabel('Time(s)', fontsize = 30)
    axes[1].set_ylabel('Node index', fontsize = 30)
    axes[1].set_aspect(0.1)
    axes[1].tick_params(axis='x', labelsize=20)
    axes[1].tick_params(axis='y', labelsize=20)
    cax_o = fig.add_axes([axes[0].get_position().x1+0.01,axes[0].get_position().y0,0.02,axes[0].get_position().height])
    cax_t = fig.add_axes([axes[1].get_position().x1+0.01,axes[1].get_position().y0,0.02,axes[1].get_position().height])
    cbar1 = plt.colorbar(im_o, cax=cax_o)
    cbar2 = plt.colorbar(im_t, cax=cax_t)
    cbar1.ax.set_ylabel(ylabel, fontsize=30)
    cbar1.ax.tick_params(axis='y', labelsize=20)
    cbar2.ax.set_ylabel(ylabel, fontsize=30)
    cbar2.ax.tick_params(axis='y', labelsize=20)
#     plt.savefig(‘./fig/lamall_RNN_{}.png’.format(suffix), bbox_inches=‘tight’)


In [5]:
import pickle

import torch
from tqdm import tqdm

In [ ]:
spike = pickle.load(open('./data/LNP_spk_all.pickle', 'rb'))

In [ ]:
spike.shape

In [ ]:
lam = pickle.load(open('./data/LNP_lam_all.pickle', 'rb'))

In [ ]:
lam.shape

In [6]:
from torch_geometric.data import Data, Batch, Dataset, InMemoryDataset

In [7]:
class Dataset(InMemoryDataset):
    def __init__(self, root=None, transform=None, pre_transform=None):
        super(Dataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
        self.config = config

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['train_n100.pt']

    
    def download(self):
        pass

    def process(self):
        lam = pickle.load(open('./data/LNP_lam_all.pickle', 'rb'))
        spike = pickle.load(open('./data/LNP_spk_all.pickle', 'rb'))

        #[tsteps, neurons] > [neurons, tsteps]
        data = spike[:1000]
        lam = lam[:1000]

        num_neurons = data.shape[0]
        total_time = data.shape[-1]
        
        time_steps = 200 #previous time steps = 20ms
        pred_steps = 20 #steps to predict
        window_size = time_steps + pred_steps - 1 # for training only
        batch_size = int(np.floor(total_time / (window_size + 1)) - 1)

        fully_connected = np.ones((num_neurons, num_neurons)) - np.eye(num_neurons)
        encoder_edge = np.where(fully_connected)
        encoder_edge = np.array([encoder_edge[0], encoder_edge[1]], dtype=np.int64)

        data = torch.FloatTensor(data)
        lam = torch.FloatTensor(lam)
        encoder_edge = torch.LongTensor(encoder_edge)

        data_list = []
        
        for i in tqdm(range(batch_size)):
            step = i * (window_size+1)
            data_sample = data[:, step:step+window_size]
            lam_tar = lam[:, step+time_steps:step+time_steps+pred_steps]
            spk_tar = data[:, step+time_steps:step+time_steps+pred_steps]
            lam_spk_tar = torch.stack([lam_tar, spk_tar], dim=-1)
            data_item = Data(x=data_sample, edge_index=encoder_edge, y=lam_spk_tar)
            data_list.append(data_item)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [10]:
dataset = Dataset('./data')

Processing...
100%|███████████████████████████████████| 21817/21817 [00:03<00:00, 7021.59it/s]
Done!


In [11]:
dataset

Dataset(21817)